In [1]:
import sys
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

import glob

In [2]:
path = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/prediction/varimp_output/'
files = glob.glob(f'{path}*')

sample1_paths = []
sample2_paths = []

for file_path in files:
    if 'sample1' in file_path:
        sample1_paths.append(file_path)
    elif 'sample2' in file_path:
        sample2_paths.append(file_path)

In [3]:
weighting_path = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/prediction/varimp_boruta_fts/'

path_list = glob.glob(f'{weighting_path}*')

string_to_remove = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/prediction/varimp_boruta_fts/'

# Iterate through the paths and perform the removal
new_path_list = []
for path in path_list:
    # Remove the specified string
    new_path = path.replace(string_to_remove, '')
    
    # Remove '.csv' from the end of the string
    if new_path.endswith('.csv'):
        new_path = new_path[:-4]
    
    new_path_list.append(new_path)
    
pull = ['CommonEF', 'Intelligence', 'UpdatingSpecific', 'pc1_new_r', 'pc2_new_r', 'pc3_new_r']

# Use a list comprehension to filter 'path_list'
filtered_path_list = [path for path in  new_path_list if path in pull]

In [4]:
import re

file_path = "/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/prediction/varimp_output/pc1_new_r_varimp_sample1_537_permimp.csv"

# Use regular expression to extract the number
match = int(re.search(r'_(\d+)_permimp\.csv', file_path).group(1))


In [5]:
def compileit(files, sample):
    import re
    performance_list = []
    for filename in files:
        performance_df = pd.read_csv(filename).iloc[:, 1:]
        string_remove = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/prediction/varimp_output/'
        performance_df = (performance_df
                          .assign(Measure = filename.replace(string_remove, '').split('_varimp')[0],
                                  Sample = sample,
                                  itr = int(re.search(r'_(\d+)_permimp\.csv', filename).group(1)))
                         )
        performance_list.append(performance_df)


    # Concatenate the DataFrames in performance_list
    concatenated_df = pd.concat(performance_list)
    
    import statsmodels.api as sm
    
    model = sm.OLS.from_formula('VIM_temp ~ Measure', data=concatenated_df).fit()
    concatenated_df['Residualized_VIM_temp'] = model.resid
    
    return concatenated_df 

sample1_concatenated_df = compileit(sample1_paths, 'sample1')
sample2_concatenated_df = compileit(sample2_paths, 'sample2')

In [6]:
#var_rank = rank/number of occurences
#mean_rank = rank amongst cog

In [7]:
def get_top_vars(concatenated_df, col):
    # Group by 'Variable' and calculate mean and std for 'Vim_temp'
    result_df = (concatenated_df
                 .groupby('Variable')[col]
                 .agg(['mean', 'std'])
                 .sort_values('mean', ascending=False)
                 #.iloc[:15]
                )

    # result_df will contain the mean and std for 'Vim_temp' for the top 15 'Variable' groups
    top_vars = result_df.index.to_list()
    custom_order = pd.CategoricalDtype(categories=top_vars, ordered=True)

    # Use the custom sorting order to sort the DataFrame
    sorted_df = concatenated_df.query('Variable in @top_vars')
    sorted_df['Variable'] = sorted_df['Variable'].astype(custom_order)
    sorted_df = sorted_df.sort_values(by='Variable')

    
    return result_df, top_vars, custom_order, sorted_df

sample1_result_df, sample1_top_vars, sample1_custom_order, sample1_sorted_df = get_top_vars(sample1_concatenated_df, 'Residualized_VIM_temp')
sample2_result_df, sample2_top_vars, sample2_custom_order, sample2_sorted_df = get_top_vars(sample2_concatenated_df, 'Residualized_VIM_temp')

In [8]:
samples_sorted = pd.concat([sample1_sorted_df, sample2_sorted_df])
samples_sorted.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/prediction/varimp_compiled_sorted_samples.csv', index=False)

In [9]:
max(samples_sorted.query('Measure == "predmeditation"').itr)

562